In [ ]:
from pathlib import Path
from PIL import Image

# =========================================================
# YOLO 라벨(.txt)을 기반으로 원본 이미지에서 객체(Insect)를 크롭하여 저장하는 스크립트
# =========================================================

# === 경로 설정 (환경에 맞게 수정 필요) ===
IMG_DIR = Path(r"C:\dataset\mealworm\images\train")   # 원본 이미지 폴더
LBL_DIR = Path(r"C:\dataset\mealworm\labels\train")   # YOLO 라벨 폴더
OUT_DIR = Path(r"C:\dataset\mealworm\crops")          # 크롭된 이미지 저장 경로

# 결과 저장 디렉토리 생성
OUT_DIR.mkdir(parents=True, exist_ok=True)

# 지원할 이미지 확장자 목록
IMG_EXTS = [".jpg", ".jpeg", ".png", ".bmp"]

# 라벨 파일 순회
for lbl_path in LBL_DIR.glob("*.txt"):
    stem = lbl_path.stem

    # 1. 대응되는 이미지 파일 찾기
    img_path = None
    for ext in IMG_EXTS:
        p = IMG_DIR / f"{stem}{ext}"
        if p.exists():
            img_path = p
            break

    if img_path is None:
        print(f"[경고] 이미지를 찾을 수 없습니다: {stem}")
        continue

    # 2. 이미지 로드 (RGB 모드)
    img = Image.open(img_path).convert("RGB")
    W, H = img.size

    # 3. 라벨 파일 읽기
    with open(lbl_path, "r", encoding="utf-8") as f:
        lines = [line.strip() for line in f.readlines() if line.strip()]

    for i, line in enumerate(lines):
        parts = line.split()
        # YOLO 포맷 확인: <class> <x_center> <y_center> <w> <h>
        if len(parts) != 5:
            print(f"[경고] 잘못된 라벨 형식: {lbl_path} -> {line}")
            continue

        cls_id, xc, yc, w, h = map(float, parts)

        # 4. 정규화된 좌표를 픽셀 좌표로 변환
        x1 = int((xc - w / 2) * W)
        y1 = int((yc - h / 2) * H)
        x2 = int((xc + w / 2) * W)
        y2 = int((yc + h / 2) * H)

        # 5. 이미지 경계를 벗어나지 않도록 클리핑
        x1 = max(0, min(W - 1, x1))
        y1 = max(0, min(H - 1, y1))
        x2 = max(0, min(W, x2))
        y2 = max(0, min(H, y2))

        if x2 <= x1 or y2 <= y1:
            print(f"[경고] 유효하지 않은 박스 크기: {lbl_path} index {i}")
            continue

        # 6. 이미지 크롭
        crop = img.crop((x1, y1, x2, y2))

        # 클래스 ID별로 폴더를 구분하여 저장
        cls_dir = OUT_DIR / f"class_{int(cls_id)}"
        cls_dir.mkdir(parents=True, exist_ok=True)

        # 크롭 이미지 저장 (파일명_인덱스.jpg)
        save_path = cls_dir / f"{stem}_{i:02d}.jpg"
        crop.save(save_path)

        # print(f"[저장] {save_path}")